In [ ]:
# 3.2 CT图像简介
import matplotlib.pyplot as plt

fig = plt.figure()

# 绘制病人的第70张CT图像
plt.imshow(ct_scans[69], cmap='gray')

plt.show()

In [ ]:
# 4.2 MHD格式数据读取
import numpy as np
import SimpleITK as sitk

# 数据
patient_path = 'CT_MHD_0101.mhd'

# 读取数据
itk_img = sitk.ReadImage(patient_path)       # ReadImage可以读取MHD格式的数据
print("图像信息：", itk_img)

# 转换为NumPy数组
ct_scans = sitk.GetArrayFromImage(itk_img)   # GetArrayFromImage()可将读取图像信息转为Numpy数组

# 获取数组形状
print("数组的形状：", ct_scans.shape)

In [ ]:
# 4.3 多张CT图像切片可视化
import numpy as np
import matplotlib.pyplot as plt

# 画布中的子图以4行4列进行排版
fig, plots = plt.subplots(4, 4, figsize=(10, 10))

# 使用for循环选取从第一张开始每9张的切片
for i in range(0, 139, 9):
    
    # 根据各子图的索引位置进行绘制
    plots[int(i / 36), int((i % 36) / 9)].imshow(ct_scans[i],cmap='gray')

    # 隐藏每张子图的坐标轴
    plots[int(i / 36), int((i % 36) / 9)].axis('off')
    
plt.show()   

In [ ]:
# 4.4 切片厚度、像素间距和远点坐标的获取
import numpy as np
import SimpleITK as sitk

# 获取切片厚度及像素间距
spacing = np.array(list(reversed(itk_img.GetSpacing())))
print("CT图像的切片厚度及像素间距是(z, y, x)：", spacing)

#获取图像原点坐标
origin = np.array(list(reversed(itk_img.GetOrigin())))
print("CT图像的原点坐标是(z, y, x)：", origin)

In [ ]:
# 4.5 图像重采样
import numpy as np
from scipy import ndimage

rescale = [1, 1, 1]

ct_scans_shape = ct_scans.shape
print("原数组形状：", ct_scans_shape, " 切片厚度和像素间距：", spacing)

# 计算调整间距后的数组形状
resize_factor = spacing / rescale
new_real_shape = ct_scans.shape * resize_factor

# 四舍五入取整（还是float类型）
new_shape = np.around(new_real_shape)

# 根据取整后的数组形状求出缩放比例
real_resize_factor = new_shape / ct_scans.shape

# 得到调整后的间距
new_spacing = spacing / real_resize_factor

# 使用zoom函数根据比例进行缩放
new_ct_scans = ndimage.interpolation.zoom(ct_scans, real_resize_factor, mode="nearest")

new_ct_scans_shape = new_ct_scans.shape
print("新数组形状：", new_ct_scans_shape, " 切片厚度和像素间距：", new_spacing)

In [ ]:
# 4.7 CT值简介以及3D可视化
import matplotlib.pyplot as plt
from skimage import measure, feature
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.mplot3d import Axes3D

threshold = 300

# 转置CT图像数组
scans = new_ct_scans.transpose(2,1,0)

# 获取等值面
verts, faces, norm, val =  measure.marching_cubes_lewiner(scans,threshold,step_size=2)

# 创建Figure对象，添加一个新的轴类型为Axes3D
fig = plt.figure(figsize=(8, 8))
ax = Axes3D(fig)
    
# 三维多边形的集合
mesh = Poly3DCollection(verts[faces], alpha=0.7)
face_color = [0.45, 0.45, 0.75]
mesh.set_facecolor(face_color)

# 将mesh添加到坐标轴中
ax.add_collection3d(mesh)

# 设置坐标轴
ax.set_xlim(0, scans.shape[0])
ax.set_ylim(0, scans.shape[1])
ax.set_zlim(0, scans.shape[2])

plt.show()

In [ ]:
# 4.8 肺实质分割（二值化）
import numpy as np
import matplotlib.pyplot as plt

# 选取第150张切片
scan = new_ct_scans[ 149 ]

# 设置阈值生成一个二值图像
binary = scan < -320

# 绘制两张子图，按照一行两列进行排版
fig, plots = plt.subplots(1, 2)

# 绘制切片原图
plots[0].imshow(scan, cmap='gray')

# 绘制二值化后的图像
plots[1].imshow(binary, cmap='gray')

plt.show()

In [ ]:
# 4.9 肺实质分割（去除边缘区域）
import numpy as np
import matplotlib.pyplot as plt
from skimage import segmentation 

# 对上一步二值图像应用skimage.segmentation中清除边界区域的函数
cleared = segmentation.clear_border(binary)

# 创建子图
fig, plots = plt.subplots(1, 2)

# 绘制binary
plots[0].imshow(binary,cmap='gray')

# 绘制cleared
plots[1].imshow(cleared,cmap='gray')

plt.show()

In [ ]:
# 4.10 肺实质分割（连通区域分割）
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure

# 应用skimage.measure中区域标记函数进行连通区域标记
label_image = measure.label(cleared)

# 创建子图
fig, plots = plt.subplots(1,2)	

# 绘制cleared
plots[0].imshow(cleared, cmap='gray')

# 绘制label_image
plots[1].imshow(label_image, cmap='gray')

plt.show()

In [ ]:
# 4.11 肺实质分割（保留最大肺实质区域）
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure

fig, plots = plt.subplots(1, 2)

# 首先绘制label_image作为对比
plots[0].imshow(label_image, cmap='gray')

# 利用regionprops返回每个区域，area属性获每个区域包含的像素个数
areas = [r.area for r in measure.regionprops(label_image)]

# 对列表排序（升序）
areas.sort()

# 判断连通区域是否大于两个
if len(areas) > 2:
    
    # 循环每个连通区域
    for region in measure.regionprops(label_image):
        
        # 判断该区域的像素个数是否小于列表的倒数第二个值
        if region.area < areas[-2]:
            
            # coords属性获取区域中每个像素的坐标
            for coordinates in region.coords:
                
                # 根据坐标将该像素值设置为0
                label_image[coordinates[0], coordinates[1]] = 0

# 重新生成二值图像                
new_binary = label_image > 0

# 绘制new_binary
plots[1].imshow(new_binary, cmap='gray')

plt.show()

In [ ]:
# 4.12 肺实质分割（腐蚀操作）
import numpy as np
import matplotlib.pyplot as plt
from skimage import morphology

# 生成扁平、圆盘状的结构元素
selem = morphology.disk(2)

# 对new_binary进行腐蚀操作
erosioned = morphology.binary_erosion(new_binary,selem)

# 创建子图
fig, plots = plt.subplots(1, 2)

# 绘制new_binary
plots[0].imshow(new_binary, cmap='gray')

# 绘制erosioned
plots[1].imshow(erosioned, cmap='gray')

plt.show()

In [ ]:
# 4.13 肺实质分割（闭合运算）
import numpy as np
import matplotlib.pyplot as plt
from skimage import morphology

# 生成扁平、圆盘状的结构元素
selem = morphology.disk(10)

# 对erosioned进行闭合操作
closed = morphology.binary_closing(erosioned,selem)

# 创建子图
fig, plots = plt.subplots(1, 2)

# 绘制erosioned
plots[0].imshow(erosioned, cmap='gray')

# 绘制closed
plots[1].imshow(closed, cmap='gray')

plt.show()

In [ ]:
# 4.14 肺实质分割（孔洞填充）
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage import filters

# 边缘检测，找到肺实质外部轮廓
edges = filters.roberts(closed)

# 填充肺实质内的孔洞
filled = ndimage.binary_fill_holes(edges)

# 创建子图
fig, plots = plt.subplots(1, 2)

# 绘制closed
plots[0].imshow(closed, cmap='gray')

# 绘制filled
plots[1].imshow(filled, cmap='gray')

plt.show()

In [ ]:
# 4.15 肺实质分割（掩膜叠加）
import numpy as np
import matplotlib.pyplot as plt

# 获取掩膜为零的布尔数组
get_high_vals = filled == 0

# 原始切片中将掩膜为零的位置设置为-1000
scan[get_high_vals] = -1000

# 创建子图
fig, plots = plt.subplots(1, 2)

# 绘制filled
plots[0].imshow(filled, cmap='gray')

# 绘制scan
plots[1].imshow(scan, cmap='gray')

plt.show()

In [ ]:
# 4.16 肺实质分割（封装代码）
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure, morphology, segmentation, filters
from scipy import ndimage

# 定义肺实质分割函数
def get_segmented_lungs(scan, plot=False):
   
    # 绘制8个步骤的结果图像，以2行4列排版
    if plot == True:
        fig, plots = plt.subplots(2, 4, figsize=(10, 5))

    # 二值化
    binary = scan < -320
    if plot == True:
        plots[0, 0].axis('off')
        plots[0, 0].imshow(binary, cmap='gray')
    
    # 去除边缘区域
    cleared = segmentation.clear_border(binary)
    if plot == True:
        plots[0, 1].axis('off')
        plots[0, 1].imshow(cleared, cmap='gray')
    
    # 连通区域标记
    label_image = measure.label(cleared)
    if plot == True:
        plots[0, 2].axis('off')
        plots[0, 2].imshow(label_image, cmap='gray')
    
    # 保留最大两个肺实质
    areas = [r.area for r in measure.regionprops(label_image)]
    areas.sort()
    if len(areas) > 2:
        for region in measure.regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:
                    label_image[coordinates[0], coordinates[1]] = 0
    new_binary = label_image > 0
    if plot == True:
        plots[0, 3].axis('off')
        plots[0, 3].imshow(new_binary, cmap='gray')
    
    # 腐蚀操作
    selem = morphology.disk(2)
    erosioned = morphology.binary_erosion(new_binary, selem)
    if plot == True:
        plots[1, 0].axis('off')
        plots[1, 0].imshow(erosioned, cmap='gray')
    
    # 闭合操作
    selem = morphology.disk(10)
    closed = morphology.binary_closing(erosioned,selem)
    if plot == True:
        plots[1, 1].axis('off')
        plots[1, 1].imshow(closed, cmap='gray')
    
    # 孔洞填充
    edges = filters.roberts(closed)
    filled = ndimage.binary_fill_holes(edges)
    if plot == True:
        plots[1, 2].axis('off')
        plots[1, 2].imshow(filled, cmap='gray')
    
    # 掩膜叠加
    get_high_vals = filled == 0
    scan[get_high_vals] = -1000
    if plot == True:
        plots[1, 3].axis('off')
        plots[1, 3].imshow(scan, cmap='gray')

    return scan

# 任意选取一张切片并进行分割展示
one_slice = new_ct_scans[ 150 ].copy()
get_segmented_lungs(one_slice, True)

# 对重采样后的CT图像进行分割
segmented = np.asarray([ get_segmented_lungs(slice) for slice in new_ct_scans ])

In [ ]:
# 4.17 肺实质分割结果3D可视化
import matplotlib.pyplot as plt
from skimage import measure, feature
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.mplot3d import Axes3D

threshold = -1000
scans = segmented.transpose(2,1,0)

verts, faces, norm, val =  measure.marching_cubes_lewiner(scans,threshold,step_size=2)

# 创建Figure对象，添加一个新的轴类型为Axes3D
fig = plt.figure(figsize=(8, 8))
ax = Axes3D(fig)
    
# 三维多边形的集合
mesh = Poly3DCollection(verts[faces], alpha=0.7)
face_color = [0.45, 0.45, 0.75]
mesh.set_facecolor(face_color)
ax.add_collection3d(mesh)

# 设置坐标轴
ax.set_xlim(0, scans.shape[0])
ax.set_ylim(0, scans.shape[1])
ax.set_zlim(0, scans.shape[2])

plt.show()

In [ ]:
# 4.19 CT图像坐标系转换
import numpy as np

# 定义坐标系转换函数，传入参数为待转换世界坐标、原点坐标、各轴间距
def world_2_voxel(world_coordinates, origin, spacing):
    
    # 将世界坐标系转换为图像坐标系
    voxel_coordinates = np.absolute(world_coordinates - origin)/spacing
    
    return voxel_coordinates

# 以图像原点坐标为例
origin_to_voxel = world_2_voxel(origin,origin,new_spacing)

print("世界坐标系的原点坐标(z,y,x)：",origin)
print("转换为图像坐标系的原点坐标(z,y,x)：",origin_to_voxel)

In [ ]:
# 4.20 读取标签文件获取肺结节信息
import numpy as np
import pandas as pd

# 肺结节标签文件路径
filename = "annotations.csv"

# 使用Pandas读取csv文件
cands_info = pd.read_csv(filename)

# 查看前5行信息
print(cands_info.head())

# 选取一位病人ID
patient_id = "CT_MHD_0101"

# 从DataFrame中查找该病人的肺结节信息
nodule_cands_df = cands_info[cands_info['seriesuid'] == patient_id ]

# 生成一个空列表，用于存储肺结节信息
nodule_cands = []

# 判断病人是否有肺结节
if len(nodule_cands_df) > 0:

    # 若有肺结节，则依次循环每个结节的信息
    for i in range(len(nodule_cands_df)):

        # 选取一个肺结节信息保存为Series对象
        cand = nodule_cands_df.iloc[i]
        
        # 获取肺结节中心坐标生成数组，并转换图像坐标
        cord_world = np.array([cand[3],cand[2],cand[1]])
        cords = world_2_voxel(cord_world,origin,new_spacing)
        
        # 将坐标与直径添加到列表中
        nodule_cands.append([cords, cand[4]])
        
        # 打印每个肺结节信息
        print("第{}个肺结节图像坐标（z、y、x）：{}，直径：{}。".format(i+1,cords, cand[4]))

else:
    print(patient_id, "没有肺结节")

In [ ]:
# 4.21 肺结节标注及可视化
import numpy as np
import matplotlib.pyplot as plt

# 创建全零数组作为初始标签
nodule_mask = np.zeros(segmented.shape,dtype=np.int8)

# 遍历肺结节信息列表，标注每个肺结节
for cand in nodule_cands:
    
    # 返回不小于真实半径的最小整数
    radius = np.ceil(cand[1] / 2)
    
    # 获取肺结节中心坐标（图像坐标）
    core = cand[0]
    
    # 生成列表用来循环填充区域（到结节中心的距离）
    nodule_range = [-radius + i for i in range(int(radius * 2) + 1)]

    # 对每个轴依次按nodule_range来循环，遍历立方体中每个像素
    for x in nodule_range:
        for y in nodule_range:
            for z in nodule_range:
                
                # 得到当前像素坐标
                coords = np.array((z,y,x) + core)
                
                # np.linalg.norm用来计算范数，默认为二范数，即距离
                if (np.linalg.norm(np.array((z,y,x)))) < radius:
                    
                    # 当像素到肺结节中心点坐标小于半径则将其值设置为1
                    coords = np.around(coords)
                    nodule_mask[int(coords[0]), int(coords[1]), int(coords[2])] = 1

# 展示索引为280的切片的原始数据及标签
fig, plots = plt.subplots(1, 2, figsize=(10, 10))
plots[0].imshow(segmented[280], cmap='gray')
plots[1].imshow(nodule_mask[280], cmap='gray')

In [ ]:
# 4.23 剪裁CT图片
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt

# 获取不等于背景（-1000）的像素点的各轴索引
zz, yy, xx = np.where(segmented != -1000)
# 从各轴坐标中求出最小和最大索引，此时索引组成的立方体即为裁剪区域
box = np.array([
    [np.min(zz), np.max(zz)], 
    [np.min(yy), np.max(yy)],
    [np.min(xx), np.max(xx)]])

# 将立方体扩展，每个轴两边增加5个像素单位
extendbox = np.vstack([np.max([[0, 0, 0], box[:, 0] - 5], axis=0),
                    np.min([segmented.shape, box[:,1] + 5], axis=0)]
                     ).T

# 根据索引裁剪图像
new_lung_mask = segmented[extendbox[0, 0]:extendbox[0, 1],
                        extendbox[1, 0]:extendbox[1, 1],
                        extendbox[2, 0]:extendbox[2, 1]]

new_nodule_mask = nodule_mask[extendbox[0, 0]:extendbox[0, 1],
                        extendbox[1, 0]:extendbox[1, 1],
                        extendbox[2, 0]:extendbox[2, 1]]

# 查看裁剪后CT图像的一张切片
fig = plt.figure()
plt.imshow(new_lung_mask[150], cmap='gray')

plt.show()

In [ ]:
# 4.24 缩放及扩充CT图像
import numpy as np
from scipy import ndimage

# 按0.7 的比例缩放图像
new_lung_mask = ndimage.zoom(new_lung_mask,(0.7,0.7,0.7), order=0)
new_nodule_mask = ndimage.zoom(new_nodule_mask,(0.7,0.7,0.7), order=0)
  
# 计算缩放后图像各轴长度与256的差值
y_pad = 256 - new_lung_mask.shape[1]
x_pad = 256 - new_lung_mask.shape[2]
    
# 计算y、x轴两边需填充的长度
lower_ypad = y_pad//2
upper_ypad = y_pad - lower_ypad
lower_xpad = x_pad//2
upper_xpad = x_pad - lower_xpad

# 填充肺实质图像
lung_mask_256 = np.pad(new_lung_mask,
                      ((0,0),(lower_ypad,upper_ypad),
                      (lower_xpad,upper_xpad)),
                      mode='constant',constant_values=-1000)

# 填充肺结节标签数组
nodule_mask_256 = np.pad(new_nodule_mask,
                      ((0,0),(lower_ypad,upper_ypad),
                      (lower_xpad,upper_xpad)),
                      mode='constant',constant_values=0)

    
# 打印最终数组形状
print("肺实质图像数组的形状：", lung_mask_256.shape)
print("肺结节标签数组的形状：", nodule_mask_256.shape)

In [ ]:
# 4.25 图像像素值标准化
import numpy as np

# 设置边界
MIN_BOUND = -1000.0
MAX_BOUND = 400.0

# 对图像标准化
lung_mask_256 = (lung_mask_256 - MIN_BOUND) / (MAX_BOUND - MIN_BOUND) * 255

# 将大于255 的值设置为255
lung_mask_256[lung_mask_256 > 255] = 255.

# 将小于0 的值设置为0
lung_mask_256[lung_mask_256 < 0] = 0.

# 最后设置数据类型
lung_mask_256 = lung_mask_256.astype(np.float16)

print("最大值：", lung_mask_256.max())
print("最小值：", lung_mask_256.min())

In [ ]:
# 4.26 肺结节切片的提取及保存
import numpy as np
import matplotlib.pyplot as plt

# 通过对标签切片求和来筛选出包含肺结节的切片索引
included_slices = [z for z in range(nodule_mask_256.shape[0]) if np.sum(nodule_mask_256[z]) > 0]

# 按照筛选出的索引获取这些切片
lung_mask_256 = lung_mask_256[included_slices]
nodule_mask_256 = nodule_mask_256[included_slices]

# 展示一组图片
show_slice = int(lung_mask_256.shape[0] / 2)

fig, plots = plt.subplots(1, 2, figsize=(10, 10))
plots[0].imshow(lung_mask_256[show_slice].astype(np.int16), cmap='gray')
plots[1].imshow(nodule_mask_256[show_slice], cmap='gray')

plt.show()

In [ ]:
# 5.2 数据读取
import numpy as np
import pydicom
import matplotlib.pyplot as plt
import os

# 数据所在的文件夹
file_path = './userdcm/'
one_dcm = '000089.dcm'

# 利用dcmread()读取.dcm文件
one_slice = pydicom.dcmread(file_path + one_dcm)
    
# 打印像素间距
print("像素间距：", one_slice.PixelSpacing)

# 打印切片左上的角世界坐标
print("切片左上角的世界坐标：", one_slice.ImagePositionPatient)

# 获取像素值
one_pixel_array = one_slice.pixel_array

# 可视化切片
fig = plt.figure()
plt.imshow(one_pixel_array, cmap='gray')
plt.show()

# 利用dcmread()函数读取每张切片文件并生成一个列表
slices = [pydicom.dcmread(file_path + s) for s in os.listdir(file_path)]

# 按照每个切片左上角世界坐标的z轴排序
slices.sort(key = lambda x: x.ImagePositionPatient[2])

print("该CT图像共{}张切片。".format(len(slices)))

In [ ]:
# 5.4 像素值转换CT值
import numpy as np
import pydicom

print("斜率：", one_slice.RescaleSlope)

print("截距：", one_slice.RescaleIntercept)

# 获取每张切片的数组值并堆叠为一个Numpy数组
ct_scans = np.stack([s.pixel_array for s in slices])

# 转换前
pixel_max = ct_scans.max()
pixel_min = ct_scans.min()
print("转换前像素值最大值：{}，最小值：{}".format(pixel_max, pixel_min))

# 对每张切片应用线性变换转CT值
for slice_number in range(len(slices)):
    
    # 获取截距和斜率
    intercept = slices[slice_number].RescaleIntercept
    slope = slices[slice_number].RescaleSlope
    
    # 线性转换为CT值
    ct_scans[slice_number] = slope * ct_scans[slice_number] + np.int16(intercept)

# 转换后
ct_max = ct_scans.max()
ct_min = ct_scans.min()
print("转换后CT值最大值：{}，最小值：{}".format(ct_max, ct_min))

In [ ]:
# 5.5 CT图像重采样
from scipy import ndimage
import numpy as np

rescale = [1, 1, 1]

ct_scans_shape = ct_scans.shape
print("原数组形状：", ct_scans_shape, " 切片厚度和像素间距：", spacing)

# 计算调整间距后的数组形状
resize_factor = spacing / rescale
new_real_shape = ct_scans.shape * resize_factor

# 四舍五入取整（还是float类型）
new_shape = np.around(new_real_shape)

# 根据取整后的数组形状求出缩放比例
real_resize_factor = new_shape / ct_scans.shape

# 得到调整后的间距
new_spacing = spacing / real_resize_factor

# 使用zoom函数根据比例进行缩放
new_ct_scans = ndimage.interpolation.zoom(ct_scans,real_resize_factor,mode='nearest')

new_ct_scans_shape = new_ct_scans.shape
print("新数组形状：", new_ct_scans.shape, " 切片厚度和像素间距：", new_spacing)

In [ ]:
# 5.6 肺实质分割及统一图像尺寸
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure, morphology, segmentation, filters
from scipy import ndimage

# 调用get_segmented_lungs函数对CT图像进行肺实质的分割
segmented = np.array([get_segmented_lungs(slice) for slice in new_ct_scans])

# 获取不等于背景（-1000）的像素点的各轴索引
zz, yy, xx = np.where(segmented != -1000)

# 从各轴坐标中求出最小和最大索引，此时索引组成的立方体即为裁剪区域
box = np.array([
    [np.min(zz), np.max(zz)], 
    [np.min(yy), np.max(yy)],
    [np.min(xx), np.max(xx)]])

# 将立方体扩展，每个轴两边增加5
extendbox = np.vstack([np.max([[0, 0, 0], box[:, 0] - 5], axis=0),
                np.min([segmented.shape, box[:, 1] + 5], axis=0)]
                ).T

# 根据索引裁剪图像
new_lung_mask = segmented[extendbox[0, 0]:extendbox[0, 1],
                        extendbox[1, 0]:extendbox[1, 1],
                        extendbox[2, 0]:extendbox[2, 1]]

# 按0.7 的比例缩放图像
new_lung_mask = ndimage.zoom(new_lung_mask,(0.7,0.7,0.7), order=0)

# 计算当前尺寸与256的差值
y_pad = 256 - new_lung_mask.shape[1]
x_pad = 256 - new_lung_mask.shape[2]

# 计算两边填充长度
lower_ypad = y_pad // 2
upper_ypad = y_pad - lower_ypad
lower_xpad = x_pad // 2
upper_xpad = x_pad - lower_xpad

# y，x轴按照指定长度填充背景值
lung_mask_256 = np.pad(new_lung_mask,
                      ((0,0),(lower_ypad,upper_ypad),
                      (lower_xpad,upper_xpad)),
                      mode='constant',constant_values=-1000)

# 打印最终数组形状
lung_mask_256_shape = lung_mask_256.shape
print("统一图像尺寸后数组形状：", lung_mask_256_shape)

# 选取一张展示
fig = plt.figure()
plt.imshow(lung_mask_256[149], cmap='gray')
plt.show()

In [ ]:
# 5.7 数据标准化及保存
import numpy as np

# 设置边界
MIN_BOUND = -1000.0
MAX_BOUND = 400.0

# 对图像标准化
lung_mask_256 = (lung_mask_256 - MIN_BOUND) / (MAX_BOUND - MIN_BOUND) * 255

# 将大于255 的值设置为255
lung_mask_256[lung_mask_256 > 255] = 255

# 将小于0 的值设置为0
lung_mask_256[lung_mask_256 < 0] = 0

# 最后设置数据类型
lung_mask_256 = lung_mask_256.astype(np.float16)

print("最大值：", lung_mask_256.max())
print("最小值：", lung_mask_256.min()) 

# 第六章 U-Net肺结节检测模型构建

In [ ]:
# 6.2 Tensorflow中的卷积层和激活函数
import tensorflow as tf
import numpy as np

input_exp = tf.constant(2., shape=[1, 32, 32, 1])

# 创建卷积核
filters = tf.Variable(tf.constant(1., shape=[3, 3, 1, 16]), name='kernel')

# 创建偏置项
bias = tf.Variable(tf.constant(0.,shape=[16]), name='bias')

# 进行卷积操作
conv = tf.nn.conv2d(input_exp, filters, strides=[1,1,1,1], padding='SAME')
print(conv)

# 添加偏置
conv_b = tf.nn.bias_add(conv,bias)
print(conv_b)

# 应用激活函数
conv_relu = tf.nn.relu(conv_b)

print(conv_relu)

In [ ]:
# 6.3 定义创建卷积层的函数
import tensorflow as tf

# 创建卷积层
def conv2d(x, filter_size, input_channels, filters, with_activation=True):
    with tf.name_scope("conv2d"):
        
        # 根据传入参数确定卷积核的形状
        filter_shape = [filter_size, filter_size, input_channels, filters]
        
        # 创建卷积核和偏置项
        initializer=tf.contrib.layers.xavier_initializer()        
        w = tf.Variable(initializer(filter_shape), name="kernel")
        b = tf.Variable(tf.truncated_normal([filters],stddev=0.1),name="bias")
        
        # 创建卷积操作
        conv = tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')
        
        # 添加偏置
        conv_b = tf.nn.bias_add(conv, b)
        
        # 是否应用激活函数
        if with_activation:
            
            # 返回应用激活函数后的输出
            return tf.nn.relu(conv_b)
        
        return conv_b

# 示例
input_data = tf.constant(0.1, shape=[1, 4, 4, 1])

conv1 = conv2d(input_data, 3, 1, 8)
print(conv1)

In [ ]:
# 6.4 收缩路径网络结构搭建
import tensorflow as tf

# 创建占位符
data = tf.placeholder(tf.float32, shape=[None, 256, 256])
keep_prob = tf.placeholder(tf.float32)

# 添加channel， 即为数据加一个维度。
input_data = tf.expand_dims(data, axis=-1)

# 构建网络收缩路径结构
# down_conv_blok1
conv1a = conv2d(input_data, 3, 1, 32)      # 输出：? * 256 * 256 * 32
conv1a = tf.nn.dropout(conv1a, keep_prob)   
conv1b = conv2d(conv1a, 3, 32, 32)         # 输出：? * 256 * 256 * 32
maxpool1 = tf.nn.max_pool(conv1b, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')             # 输出：? * 128 * 128 * 32

# down_conv_blok2
conv2a = conv2d(maxpool1, 3, 32, 80)       # 输出：? * 128 * 128 * 80
conv2a = tf.nn.dropout(conv2a, keep_prob)
conv2b = conv2d(conv2a, 3, 80, 80)         # 输出：? * 128 * 128 * 80
maxpool2 = tf.nn.max_pool(conv2b, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')             # 输出：? * 64 * 64 * 80

# down_conv_blok3
conv3a = conv2d(maxpool2, 3, 80, 160)      # 输出：? * 64 * 64 * 160
conv3a = tf.nn.dropout(conv3a, keep_prob)
conv3b = conv2d(conv3a,3, 160, 160)       # 输出：? * 64 * 64 * 160
maxpool3 = tf.nn.max_pool(conv3b, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')            # 输出：? * 32 * 32 * 160

# 中间卷积层
conv4a = conv2d(maxpool3, 3, 160, 320)     # 输出：? * 32 * 32 * 320 
conv4a = tf.nn.dropout(conv4a, keep_prob)
conv4b = conv2d(conv4a, 3, 320, 320)       # 输出：? * 32 * 32 * 320

print(conv4b)

In [ ]:
# 6.5 拓展路径网络结构搭建
import tensorflow as tf

# 构建网络扩展路径结构
# up_conv_blok1
up_conv1 = tf.image.resize_images(conv4b, [64, 64])    # 输出：? * 64 * 64 * 320 
concat1 = tf.concat([up_conv1, conv3b], axis=-1)    # 输出：? * 64 * 64 * 480
conv5a = conv2d(concat1, 3, 480, 160)    # 输出：? * 64 * 64 * 160
conv5a = tf.nn.dropout(conv5a, keep_prob)
conv5b = conv2d(conv5a, 3, 160, 160)    # 输出：? * 64 * 64 * 160

# up_conv_blok2
up_conv2 = tf.image.resize_images(conv5b, [128, 128])    # 输出：? * 128 * 128 * 160
concat2 = tf.concat([up_conv2,conv2b], axis=-1)                                 # 输出：? * 128 * 128 * 240
conv6a = conv2d(concat2, 3, 240, 80)    # 输出：? * 128 * 128 * 80
conv6a = tf.nn.dropout(conv6a, keep_prob)
conv6b = conv2d(conv6a, 3, 80, 80)    # 输出：? * 128 * 128 * 80

# up_conv_blok3
up_conv3 = tf.image.resize_images(conv6b, [256, 256])    # 输出：? * 256 * 256 * 80
concat3 = tf.concat([up_conv3, conv1b], axis=-1)    # 输出：? * 256 * 256 * 112
conv7a = conv2d(concat3, 3, 112, 32)                  # 输出：? * 256 * 256 * 32
conv7a = tf.nn.dropout(conv7a, keep_prob)
conv7b = conv2d(conv7a, 3, 32, 32)    # 输出：? * 256 * 256 * 32

# 输出层
predictions = conv2d(conv7b, 3, 32, 2, False)    # 输出：? * 256 * 256 * 2
print(predictions)

In [ ]:
# 6.6 损失函数、优化器及模型持久化
import tensorflow as tf

# 标签数据占位符
labels = tf.placeholder(tf.int32,shape=[None, 256, 256])

# 设置权重
weights = (200 - 1) * tf.to_float(labels) + 1

# 设置交叉熵损失函数
loss = tf.losses.sparse_softmax_cross_entropy(
        labels=labels, logits=predictions, weights=weights)

# 设置优化器
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)

# 模型的保存和恢复变量
saver = tf.train.Saver()

In [ ]:
# 6.7 定义验证函数
import tensorflow as tf

def validate(session, x, y, batch_size):
    
    # 用于存放每个batch的损失
    losses = []
    
    # 循环每个批次
    for start in range(0, len(x), batch_size):
        
        # 将每个batch数据传入占位符
        feed_dict = {}
        feed_dict[data] = x[start: start + batch_size]
        feed_dict[labels] = y[start: start + batch_size]
        
        # 在验证时不应该进行dropout，所以keep_prob设置为1
        feed_dict[keep_prob] = 1.
        
        # 目标运算
        fetches = loss
        
        # 调用session.run()计算每个batch的损失
        outputs = session.run(fetches, feed_dict)
        
        # 每个batch的loss加入列表
        losses.append(outputs)

    # 返回整个传入数据集的平均损失
    return sum(losses) / float(len(losses))

In [ ]:
# 6.8 定义训练函数
import random
import tensorflow as tf

def train(session, x_train, y_train, x_val, y_val, epochs, batch_size, train_dir, best_loss):

    # 根据回合数进行循环
    for e in range(epochs):
        print("[Epoch %s of %s]" % (e + 1, epochs))

        # 创建训练数据的索引并随机打乱
        indices = np.random.permutation(y_train.shape[0])
        
        # start为每个batch的其实索引，循环的步长为batch_size
        for start in range(0, len(x_train), batch_size):
            
            # 将一个batch数据传入占位符
            feed_dict = {}
            feed_dict[data] = x_train[indices[start: start + batch_size]]
            feed_dict[labels] = y_train[indices[start: start + batch_size]]
            
            # 在训练时要应用dropout，这里设置keep_prob为0.8
            feed_dict[keep_prob] = 0.8
            
            # 目标运算
            fetches = optimizer
            
            # 运行计算
            session.run(fetches, feed_dict)
        
        # 获取训练集上的损失
        train_loss = validate(session, x_train, y_train, batch_size)
        
        # 获取验证集上的损失
        val_loss = validate(session, x_val, y_val, batch_size)
        print("训练集损失：%s；验证集损失：%s" % (train_loss, val_loss))
        
        # 如果损失达到了设定的损失值，则将当前模型参数保存。
        if val_loss <= best_loss:
            print("新的最小验证集损失：%s，保存模型!" % (val_loss))
            
            # 将当前最小损失赋值给best_val_loss
            best_loss = val_loss

            # 保存模型
            saver.save(session, train_dir + 'model_unet')

In [ ]:
# 6.9 数据集的读取与划分
import tensorflow as tf
import numpy as np
import os
from sklearn.model_selection import train_test_split

# 设置数据路径
data_dir = './data_dir/'
labels_dir = './labels_dir/'

# 获取路径中的文件列表
data_files = os.listdir(data_dir)
data_files.sort()

# 创建保存数据的列表
x = []
y = []

# 遍历文件列表中的每个文件
for num in range(len(data_files)):
    
    # 获取文件路径
    patient_id = data_files[num][:-14]
    data_path = data_dir + patient_id + '_lung_mask.npy'
    label_path = labels_dir + patient_id + '_nodule_mask.npy'
    
    # 加载文件并添加列表中
    x.append(np.load(data_path))
    y.append(np.load(label_path))

print("数据量：%s 个病人" %  len(x))

# 利用np.concatenate将列表内的数组拼接为一个数组
x = np.concatenate(x).astype(np.float32)
y = np.concatenate(y).astype(np.int8)

print("总切片数：", x.shape[0])

# 划分训练集和验证集
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size = 0.3,random_state=1024)

print("训练集：{}；验证集：{}".format(x_train.shape[0], x_val.shape[0]))

In [ ]:
# 6.10 模型训练
import tensorflow as tf

# 设置模型保存路径
train_dir = './train_dir/'

# 在上下文语境中创建TensorFlow会话
with tf.Session() as sess:
    
    if os.path.exists(train_dir):
               
        # 加载模型参数
        saver.restore(sess, train_dir + "model_unet_loss")
    else:
        print("创建新的模型！")
        
        # 初始化所有参数
        tf.global_variables_initializer()
        
    # 获取所有可训练变量并打印
    parameter_num = sum(v.shape.num_elements() for v in tf.trainable_variables())
    print('模型可训练参数总量：%d' % parameter_num)
        
    # 调用train函数设置超参数进行训练
    train(sess, x_train, y_train, x_val, y_val, epochs=5, batch_size=4, train_dir=train_dir, best_loss=1)

# 第七章 肺结节检测与筛选

In [ ]:
# 7.2 定义检测函数
import tensorflow as tf
import numpy as np

# 应用softmax
predict_softmax = tf.nn.softmax(predictions)

def predict(session, x, batch_size):
    
    # 创建全0标签
    result = np.zeros((x.shape[0], 256, 256))
    
    # 按批次进行计算
    for start in range(0, len(x), batch_size):
        
        # 将每个batch数据传入占位符
        feed_dict = {}
        feed_dict[data] = x[start: start + batch_size]
        
        # 在检测时不应该进行dropout，所以keep_prob设置为1
        feed_dict[keep_prob] = 1

        fetches = predict_softmax
        
        # 获取神经网络的输出
        outputs = session.run(fetches, feed_dict)
        
        # 获取类别为1的概率
        result[start:start + batch_size] = outputs[:,:,:,1]

    return result.astype(np.float16)

In [ ]:
# 7.3 数据读取与肺结节检测
import tensorflow as tf
import numpy as np
import os

train_dir = './train_dir/'
predict_data_dir = './predict_data_dir/'

with tf.Session() as sess:
        
    # 加载模型
    saver.restore ( sess, train_dir + "model_unet_loss")
    
    data_list = os.listdir(predict_data_dir)
    data_list.sort()
    print("数据量：%s 个病人" %  len(data_list))
    
    # 遍历每个病人数据应用模型
    for num in range(len(data_list)):
        
        # 加载数据
        path = predict_data_dir + data_list[num]
        x = np.load(path)
        
        patient_id = data_list[num][:-14]
        print("正在检测：%s" % patient_id)
        
        # 调用predicat进行肺结节检测
        result = predicat(sess, x, 8)

        print("成功检测：%s" % patient_id)

In [ ]:
# 7.4 检测结果可视化
import matplotlib.pyplot as plt
import numpy as np

# 加载数据
data_one = np.load("CT_DICOM_0009_lung_mask.npy").astype(np.int16)
result_one = np.load("CT_DICOM_0009.npy")

# 设置展示的切片索引
show_slice = 100

# 绘图展示
fig, plots = plt.subplots(1, 2, figsize=(10,10))
plots[0].imshow(data_one[show_slice], cmap='gray')

# 网络输出的像素值是该像素是肺结节的概率，因此以0.5为阈值转换为0和1。
plots[1].imshow(result_one[show_slice] > 0.5, cmap='gray')

In [ ]:
# 7.6 初始化候选块
import numpy as np

def init_candidate_cubes(result_label):
    
    # 创建候选块存放列表
    candidate_cubes = []
    
    # 初始化四个候选块为CT图像的角
    init_cube1 = result_label[:32, :32, :32]
    init_cube2 = result_label[:32, 224:, :32]
    init_cube3 = result_label[:32, 224:, 224:]
    init_cube4 = result_label[:32, :32, 224:]
    
    # 将四个候选块添加到列表中
    candidate_cubes.append((np.sum(init_cube1), 0, 0, 0))
    candidate_cubes.append((np.sum(init_cube2), 0, 224, 0))
    candidate_cubes.append((np.sum(init_cube3), 0, 224, 224))
    candidate_cubes.append((np.sum(init_cube4), 0, 0, 224))
    
    return candidate_cubes

# 示例
result_label = np.load('result_label_0001.npy')
candidate_cubes = init_candidate_cubes(result_label)
for i in range(len(candidate_cubes)):
    print("初始候选块{}：{}".format(i + 1, candidate_cubes[i]))

In [ ]:
# 7.7 候选块的更新规则
import numpy as np

def updata_candidate_cube(candidate_cubes, test_score, test_location):
    num_overlapping = 0
    overlapping_cube = None
    
    # 判断当前新候选块与列表中的候选块是否有交集
    for cube in candidate_cubes:
        cube_location = np.array(cube[1:])
        
        # 当所有轴的坐标之差都小于等于32，则存在交集
        if np.all(cube_location - test_location <= 32):
            num_overlapping += 1
            if num_overlapping > 1:
                return  
            overlapping_cube = cube
            
    new_cube = (test_score, test_location[0], test_location[1], test_location[2])
    
    # 如果都没有交集则直接去掉列表中概率最小的候选块 并将当前候选块添加进去
    if num_overlapping == 0: 
        candidate_cubes.remove(min(candidate_cubes, key = lambda t : t[0]))
        candidate_cubes.append(new_cube)
    
    # 如果有一个有交集的
    if num_overlapping == 1: 
        
        # 并且新的立方体概率值更大，则移除与之存在交集的立方体插入新的候选块
        if overlapping_cube[0] < test_score:
            
            candidate_cubes.remove(overlapping_cube)
            candidate_cubes.append(new_cube)

In [ ]:
# 7.8 筛选候选块
import numpy as np

def find_candidate_cubes(result_label):  
    
    # 防止数据不稳定
    result_label /= np.amax(result_label) 
    
    # 初始化候选块
    candidate_cubes = init_candidate_cubes(result_label)
    
    # 初始化最小概率值为0
    min_score = 0
    
    # 遍历标签数据，y、x轴步长为4
    for i in range(result_label.shape[0] - 31):
        for j in range(0, 256 - 31, 4):
            for k in range(0, 256 - 31, 4):
                
                # 获取当前立方体
                test_cube = result_label[i: i + 32, j: j + 32, k: k + 32]
                
                # 计算概率（求和）
                test_score = np.sum(test_cube)
                
                # 如果大于当前候选块中最小的概率值则更新候选块
                if(test_score > min_score):
                    updata_candidate_cube(candidate_cubes, test_score, np.array((i, j, k)))
                    
                    # 重新设置当前概率的最小值
                    min_score = min(candidate_cubes, key = lambda t : t[0])[0]
    
    # 按照概率进行排序                
    sorted_candidate_cubes = sorted(candidate_cubes, key=lambda tup : tup[0], reverse=True)

    return sorted_candidate_cubes 

In [ ]:
# 7.9 候选块的筛选与拼接
import matplotlib.pyplot as plt
import numpy as np
import os
  
# 加载原始图像与标签数据,数据类型转换为float32
result_label = np.load('./unet_result_dir/CT_DICOM_0009.npy').astype(np.float32)
origin_data = np.load('./predict_data_dir/CT_DICOM_0009_lung_mask.npy').astype(np.float32)

# 调用函数获取候选块列表
candidate_cubes = find_candidate_cubes(result_label)

# 获取原图像
origin_data_cubes = []
for i in range(4):
    cube = candidate_cubes[i]

    # 根据候选块的坐标位置获取原图像
    origin_data_cubes.append(origin_data[cube[1]:cube[1] + 32,
                       cube[2]:cube[2] + 32, cube[3]:cube[3] + 32])

# 对原图像的立方体块进行拼接
combined_cubes_top = np.concatenate(
    [origin_data_cubes[0], origin_data_cubes[1]], axis=2) 
combined_cubes_bottom = np.concatenate(
	 [origin_data_cubes[2], origin_data_cubes[3]], axis=2)
combined_cubes = np.concatenate(
    [combined_cubes_top, combined_cubes_bottom], axis=1)

# 绘制拼接结果的单张切片
plt.imshow(combined_cubes[10],cmap="gray")
plt.show()

# 第八章 肺癌诊断神经网络模型构建

In [ ]:
# 8.2 定义创建网络结构的函数
import tensorflow as tf
import numpy as np

# 创建权重函数
def weight_variable(shape, name="kernel"):
    
    # 创建初始化器
    initializer = tf.contrib.layers.xavier_initializer()
    return tf.Variable(initializer(shape), name=name)

# 创建偏置函数
def bias_variable(shape, name="bias"):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

# 3D卷积
def conv3d_with_bn(x, filter_size, input_channels, filters, is_training):
    with tf.name_scope("conv3d"):
        shape = [filter_size, filter_size, filter_size, input_channels, filters]
        w = weight_variable(shape)
        b = bias_variable([filters])
        
        # 利用tf.nn.conv3d进行3D卷积操作
        conv_3d = tf.nn.conv3d(x, w, strides=[1, 1, 1, 1, 1], padding='SAME')
        conv_3d_b = tf.nn.bias_add(conv_3d, b)
        relu = tf.nn.relu(conv_3d_b)
        
        # 应用batch_normalization
        bn = tf.layers.batch_normalization(relu, training=is_training)
        return bn

# 全连接
def new_fc(x, num_inputs, num_outputs):
    w = weight_variable([num_inputs, num_outputs])
    b = bias_variable([num_outputs])

    fc = tf.matmul(x, w + b)

    return fc

# 3D池化
def max_pool3d(x, ksize):
    return tf.nn.max_pool3d(x, ksize=[1, ksize[0], ksize[1], ksize[2], 1], strides=[
        1, ksize[0], ksize[1], ksize[2], 1], padding='VALID')

def avg_pool3d(x, ksize):
    return tf.nn.avg_pool3d(x, ksize=[1, ksize[0], ksize[1], ksize[2], 1], strides=[
        1, ksize[0], ksize[1], ksize[2], 1], padding='VALID')

# 示例
input_data = tf.constant(0.1, shape=[1, 4, 4, 4, 1])

conv3d1 = conv3d_with_bn(input_data, 3, 1, 8, True)
print(conv3d1)

maxpool1 = max_pool3d(conv3d1, [2, 2, 2])
print(maxpool1)

reshaped = tf.reshape(maxpool1, [-1, 8])
fc1 = new_fc(reshaped, 8, 2)
print(fc1)

In [ ]:
# 8.3 3D CNN网络结构的搭建
import tensorflow as tf

# 定义占位符，设置data的name为data
data = tf.placeholder(tf.float32, shape=[None, 32, 64, 64], name='data')
is_training = tf.placeholder(tf.bool, name='is_training')

# 增加维度（channel）
input_data = tf.expand_dims(data,axis=-1)     # ? * 32 * 64 * 64 * 1

avgpool1 = avg_pool3d(input_data, [2, 1, 1])     # ? * 16 * 64 * 64 * 1

conv1 = conv3d_with_bn(avgpool1, 3, 1, 32, is_training)     # ? * 16 * 64 * 64 * 32
maxpool1 = max_pool3d(conv1, [1, 2, 2])     # ? * 16 * 32 * 32 * 32

conv2 = conv3d_with_bn(maxpool1, 3, 32, 64, is_training)     # ? * 16 * 32 * 32 * 64
maxpool2 = max_pool3d(conv2, [2, 2, 2])     # ? * 8 * 16 * 16 * 64

conv3 = conv3d_with_bn(maxpool2, 3, 64, 64, is_training)     # ? * 8 * 16 * 16 * 64
maxpool3 = max_pool3d(conv3, [2, 2, 2])      # ? * 4 * 8 * 8 * 64

conv4 = conv3d_with_bn(maxpool3, 3, 64, 128, is_training)    # ? * 4 * 8 * 8 * 128
maxpool4 = max_pool3d(conv4, [2, 2, 2])     # ? * 2 * 4 * 4 * 128

conv5 = conv3d_with_bn(maxpool4, 3, 128, 64, is_training)    # ? * 2 * 4 * 4 * 64
maxpool5 = max_pool3d(conv5, [2, 2, 2])     # ? * 1 * 2 * 2 * 64

conv6 = conv3d_with_bn(maxpool5, 3, 64, 1, is_training)    # ? * 1 * 2 * 2 * 1

flatten = tf.reshape(conv6, [-1, 4])      # ? * 4

predictions = new_fc(flatten, 4, 2)      # ? * 2

# 添加到集合中在重新加载模型时方便获取
tf.add_to_collection('predict', predictions)

print(predictions)

In [ ]:
# 8.4 损失函数、优化器及模型持久化
import tensorflow as tf

labels = tf.placeholder(tf.int32, shape=[None])

# 设置交叉熵损失函数
loss = tf.losses.sparse_softmax_cross_entropy(labels, predictions)

# 用tf.layers.batch_normalization时需要设置
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    
    # 设置优化器
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

# 模型的保存和恢复变量
saver = tf.train.Saver()

In [ ]:
# 8.5 定义评估函数
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix

def accuracy(session, x, y, batch_size):
    
    # losses用来存储每个batch的loss，y_pred用于存储预测类别
    losses = []
    y_pred = np.array([])
    
    # 分批次进行评估
    for start in range(0, len(x), batch_size):
        x_batch = x[start: start + batch_size]
        y_batch = y[start: start + batch_size]
        
        # 将数据喂给占位符
        feed_dict = {}
        feed_dict[data] = x_batch
        feed_dict[labels] = y_batch
        
        # 评估过程中为预测模式
        feed_dict[is_training] = False
        
        fetches = [loss, tf.nn.softmax(predictions)]

        outputs = session.run(fetches, feed_dict)
        
        # loss操作的返回值是一个数值，需要首先转化为列表
        losses += [outputs[0]]
        
        # 获取模型分类结果
        y_pred = np.append(y_pred, np.argmax(outputs[1], axis=1))
        
   # 获取混淆矩阵及TP、FP、TN、FN
    TN, FP, FN, TP = confusion_matrix(y, y_pred).ravel()

    # 计算正确率、敏感度、特异度
    acc = (TP + TN) / (TP + FP + TN + FN)
    sens = (TP) / (TP + FN)
    spec = (TN) / (FP + TN)
    
    # 计算平均损失
    avg_loss = sum(losses) / len(losses)

    return avg_loss, acc, sens, spec

In [ ]:
# 8.6 数据集的读取与划分
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

# 设置文件路径
data_dir = './data_dir'
labels_file = 'labels.csv'

patient_data = os.listdir(data_dir)
patient_data.sort()

labels_df = pd.read_csv(labels_file)
print(labels_df.head())

x = []
y = []

# 读取数据并获取标签存储到列表中
for file in patient_data:
    
    # 获取文件路径 加载数据
    path = os.path.join(data_dir, file)
    image = np.load(path)
    
    pat_id = file[:-4]
    
    # 获取对应的标签
    row = labels_df[labels_df['id'] == pat_id]
    if len(row) == 1:
        label = int(row['cancer'])
        x.append(image)
        y.append(label)
    else:
        print(pat_id, "无标签！")
        
x = np.asarray(x, dtype=np.float32)
y = np.asarray(y, dtype=np.int32)
num_total = x.shape[0]
print("数据量：%s " %  num_total)

# 数据集划分
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1024, stratify=y)

print("训练集：{}；测试集：{}".format(x_train.shape[0], x_test.shape[0]))

In [ ]:
# 8.7 训练模型
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split

# 设置模型保存路径
train_dir = './cnn3d_weights/'
epochs=5
batch_size=8

# 在上下文语境中创建TensorFlow会话
with tf.Session() as sess:
    
    if os.path.exists(train_dir):
               
        # 加载模型参数
        saver.restore(sess, train_dir + 'cnn3d')
    else:
        print("创建新的模型！")
        
        # 初始化所有参数
        sess.run(tf.global_variables_initialier())
        
        # 获取所有可训练变量并打印
        parameter_num = ____
        print('模型可训练参数总量：%d' % parameter_num)

    # 根据回合数进行循环
    for e in range(epochs):
        print("=" * 80)
        print("[Epoch %s of %s]" % (e + 1, epochs))

        # 创建训练数据的索引并随机打乱
        indices = np.random.permutation(y_train.shape[0])
        
        # start为每个batch的其实索引，循环的步长为batch_size
        for start in range(0, len(x_train), batch_size):
            input_feed = {}
            input_feed[data] = x_train[indices[start:start + batch_size]]
            input_feed[labels] = y_train[indices[start:start + batch_size]]
            input_feed[is_training] = True
            
            output_feed = optimizer

            sess.run(output_feed, input_feed)
        
        # 在一个回合结束后在训练集和测试集上进行验证
        train_loss, train_acc, train_sens, train_spec = accuracy(sess, x_train, y_train, batch_size)
        print("训练集损失：{:.2f}；正确率:{:.2f}；敏感度：{:.2f}；特异度：{:.2f}".format(train_loss, train_acc, train_sens, train_spec))
        
        val_loss, val_acc, val_sens, val_spec = ____
        print("测试集损失：{:.2f}；正确率:{:.2f}；敏感度：{:.2f}；特异度：{:.2f}".format(val_loss, val_acc, val_sens, val_spec))

# 第九章 肺癌诊断模型效果评估

In [ ]:
# 9.2 模型评估
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns

def accuracy_predict(session, x, y, batch_size, show_confusion):
    
    # 创建评价指标初始化为0， losses用来存储每个batch的loss
    losses = []
    y_prob = np.array([])
    
    # 分批次进行评估
    for start in range(0, len(x), batch_size):
        x_batch = x[start: start + batch_size]
        y_batch = y[start: start + batch_size]
        
        # 将数据喂给占位符
        input_feed = {}
        input_feed[data] = x_batch
        input_feed[labels] = y_batch
        
        # 评估过程中不需要进行权重更新
        input_feed[is_training] = False
        
        output_feed = [loss, tf.nn.softmax(predictions)]

        outputs = session.run(output_feed, input_feed)
        
        losses += [outputs[0]]
        
        # 获取正例的概率
        y_prob = np.append(y_prob, outputs[1][:,1])

    # 获取混淆矩阵及TP、FP、TN、FN
    conf_matrix = confusion_matrix(y, y_prob > 0.5)
    
    # 转换为DataFrame并打印
    if show_confusion:
        
        ## 设置正常显示中文
        sns.set(font='SimHei')
        
        ## 绘制热力图
        ax = sns.heatmap(conf_matrix, annot=True, fmt='d', 
                 xticklabels=["阴性(0)","阳性(1)"],
                 yticklabels=["阴性(0)","阳性(1)"])
        
        ax.set_ylabel('真实标签')
        ax.set_xlabel('预测标签')
        ax.set_title('混淆矩阵热力图')
    
    # 计算正确率、敏感度、特异度
    TN, FP, FN, TP = conf_matrix.ravel()
    acc = (TP + TN) / (TP + FP + TN + FN)
    sens = (TP) / (TP + FN)
    spec = (TN) / (FP + TN)
    
    # 计算平均损失
    avg_loss = sum(losses) / len(losses)

    return avg_loss, acc, sens, spec, y_prob

# 重新创建会话加载模型
with tf.Session() as sess:

    # 加载保存的最小损失模型
    saver.restore(sess, train_dir + 'cnn3d')

    # 获取模型在测试集上的性能
    avg_loss, acc, sens, spec, probs = accuracy_predict(sess, x_test, y_test, 8, True)
    print("测试集正确率：{:.2f}".format(acc))
    print("测试集敏感度：{:.2f}".format(sens))
    print("测试集特异度：{:.2f}".format(spec))


In [ ]:
# 9.3 绘制ROC曲线图并计算AUC值
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# 根据测试集的实际标签和模型输出概率获取roc曲线
fpr,tpr, _ = roc_curve(y_test, probs)

# 计算auc
roc_auc = auc(fpr, tpr)

print("AUC为：{:.2f}".format(roc_auc))

# 绘图
fig = plt.figure()

# 根据for和tpr绘制roc图像
plt.plot(fpr, tpr, color='darkorange')

# 绘制对角线
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')

# 设置坐标轴
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('FPR')
plt.ylabel('TPR')

# 设置标题
plt.title('ROC曲线')

plt.show()

In [ ]:
# 9.4 肺癌诊断流程
import tensorflow as tf
import numpy as np

def predict_class(x):
    
    train_dir = './cnn3d_weights/'
    
    predict_data = np.array([x])
    
    with tf.Session() as sess:
        
        # 加载计算图，文件名：cnn3d.meta
        new_saver = tf.train.import_meta_graph(train_dir + 'cnn3d.meta')
        
        # 加载模型参数
        new_saver.restore(sess, train_dir + 'cnn3d')
        
        # 获取默认的计算图
        graph = tf.get_default_graph()
        
        # 获取用到的张量
        data = graph.get_tensor_by_name('data:0')
        is_training = graph.get_tensor_by_name('is_training:0')
        predictions = tf.get_collection('predict')
        
        feed_dict = {}
        feed_dict[data] = predict_data
        feed_dict[is_training] = False
        
        fetches= tf.nn.softmax(predictions)
        
        output = sess.run(fetches, feed_dict)
        
        return np.argmax(output)

# 加载测试数据
test_data = np.load("test.npy")

# 获取预测结果
result = predict_class(test_data)

print("诊断结果：", "阳性" if result == 1 else "阴性")